In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType,DoubleType,DecimalType
from pyspark.sql.functions import * 

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext

sc = SparkContext.getOrCreate()
gc = GlueContext(sc)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
12,application_1642141202245_0013,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Create dynamic frame
source_df = gc.create_data_frame_from_catalog( 
    database = "trucapedb", 
    table_name = "trucape_invoices", 
)

# source_dyf = gc.create_data_frame.from_options(
#     connection_type='s3',
#     connection_options={'paths': ["s3://sol-dev-source/Trucape-Invoices"]},
#     format='csv,'
# )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
source_df.show()

In [ ]:
source_df.printSchema()

In [10]:
# ADD YEAR TO DUE DATE
# source_df_with_date_format = source_df.withColumn("due date", F.concat(col("due date"),F.lit("-2022"))) \
#     .show()

#  ADD YEAR TO ISSUE DATE
# source_df_with_date_format = source_df.withColumn("issue date", F.concat(col("issue date"),F.lit("-2022"))) \
#     .show()

# add year to columns: due date & isue date 
source_df = source_df.withColumn("due date", F.concat(col("due date"),F.lit("-2022"))) \
    .withColumn("issue date", F.concat(col("issue date"),F.lit("-2022"))) \
    .show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+--------------------+------------+--------+-------------+------------+-----------+----------+
|external reference|number|        counterparty|  amount due|currency|cost currency|costing rate|   due date|issue date|
+------------------+------+--------------------+------------+--------+-------------+------------+-----------+----------+
|          SC25505A|137113|   Fruit Traders LLC|  10,800.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25506A|137114|   Fruit Traders LLC|  10,800.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25503A|137116|   Fruit Traders LLC|  10,800.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25504A|137117|   Fruit Traders LLC|  10,800.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25499A|137078|   Fruit Traders LLC|  10,134.00 |     USD|          ZAR|      15.862|24-Jan-2022|3-Jan-2022|
|          SC25500A|137079|   Fr

In [15]:
# DUE DATE FORMATTING
# source_df_format = source_df1.select(col("due date"),to_date(col("due date"), "d-MMM-yyyy")) \
#   .show()

# ISSUE DATE FORMATTING
# df1 = source_df1.select(col("issue date"),to_date(col("issue date"), "d-MMM")) \
#   .show()

# FORMAT COLUMNS: DUE DATE & ISSUE DATE 
source_df = source_df.select((col("due date"),to_date(F.col("due date"), "d-MMM-yyy")) | 
                             (col("issue date"),to_date(F.col("issue date"), "d-MMM-yyy"))) \
                                .show()
      

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'NoneType' object has no attribute 'select'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'select'



In [ ]:
# # Set decimal places
# source_df_decimals = source_df.select( 
#                 F.round(source_df["amount due"],2),
#                 F.round(source_df["costing rate"],4)
# ).show()

# source_df_decimals = source_df.select(
#     round('amount due', 2).alias('r')).show()

for c in source_df.columns:
    df3 = source_df.withColumn(c, round(c, 2))
    
df3.show()

In [ ]:
# Set column types
df2 = source_df.withColumn("costing rate",col("costing rate").cast(StringType())) \
    .withColumn("due date",col("due date").cast(StringType())) \
    .withColumn("issue date",col("issue date").cast(StringType()))
df2.printSchema()
df2.show()

In [ ]:
# source_df_decimals = df2.withColumn(
#    format_string(".04d%", col( 'costing rate').cast('int')).show()
# )

source_df_decimals = df2.select(
    round('costing rate',4)).show()
 
# source_df_trailingzeros = df2.withColumn(lpad(source_df_trailingzeros.costing_rate,4, '0')).show()



In [ ]:
# Data types as they are in the file
source_df.printSchema()